# Probability

We are back to using R. However, if you feel more comfortable using Python notebooks and magics feel free to do that. 

In this lab we will be using R to calculate probabilities. Most of this is fairly straightforward 

Before we do anything, we will need to load some libraries. 

In [2]:
library(tidyverse)
library(Lahman)
library(mosaicData)

## Probabilities in R

We have been calculating probabilities in R throughout the term. We have done this by using the `p[dist]` commands where `[dist]` is some probability distribution. For instance, if we wanted to calculate the probability of $X>1$ where $X$ is a random variable with standard normal distribution then we would use `pnorm(1,lower.tail=FALSE)`

In [3]:
pnorm(1,lower.tail=FALSE)

[1] 0.1586553

This calculation tells us that there is a 15.87% chance of $X>1$ in standard normal distribution. We can do something similar with discrete distributions. For instance if we wanted to calculate the probability that at most 4 out of 10 elements of a group are "off" when the probability of being "off" in the total population is 80% we would use the binomial distribution. More specifically we would use `pbinom(4,10,.8)`

In [10]:
pbinom(4,10,.8)

[1] 0.006369382

This tells us that there is a 0.637% chance of at most 4 out of 10 elements being "off"

### Differences between Discrete and Continuous Distributions in R

As we have noted previously there are four functions for each probability distribution in R. If `[DIST]` is one of the distribution suffixes below 

- Continuous
    - `norm` : A normal distibutrion
    - `exp` : An exponential distribution
    - `lnorm` : A log-normal distribution
    - `weibull` : A Weibull distribution
- Discrete
    - `unif` : A uniform distribution
    - `pois` : A Poisson distribution
    - `binom` : A binomial distribution
    
then there are four functions associated to `[DIST]`. Each distribution has its own `parameters` that must be included in the function. For instance, if `[DIST]` is `norm` then the parameters are `mean` and `sd`. When these are not included R defaults to the standard normal distribution with `mean=0` and `sd=1`. 

The four function have different behaviour depending on whether `[DIST]` is continuous or discrete.

- Continuous
    - `d[DIST](x,parameters)` 
        - The value of the probability density function at $x$, $f(x)$
        - This value doesn't have a lot specific uses and we will rarely use it in applications
    - `p[DIST](x,parameters)` 
        - The proportion of values underneath the distribution with $X\le x$, where $X$ is the random variable. 
        - Since this is a continuous distribution, $P(X\le x)=P(X < x)$.
        - If we wanted to calculate $P(X>x)$ then we would use `p[DIST](x,parameters,lower.tail=FALSE)`
    - `q[DIST](q,parameters)`
        - The value $x$ for which $P(X \le x)=q$. 
        - Since this is a continuous distribution, $q=P(X\le x)=P(X < x)$.
        - This function is used to calculate percentiles. 
        - Again if we wanted to calculate the value of $q$ for which $P(X>x)$ then we would use `q[DIST](q,parameters,lower.tail=FALSE)`
    - `r[DIST](n,parameters)`
        - Returns a list of $n$ values distributed according to `[DIST]`

- Discrete
    - `d[DIST](x,parameters)`
        - The value of the probability function at $x$, $f(x)$
        - Since `[DIST]` is a discrete distribution, the value of $f(x)$ is equal to $P(X=x)$.
        - We use this function when we want to calculate the probability of a singular value according to a discrete distribution,
    - `p[DIST](x,parameters)`
        - The proportion of values underneath the distribution with $X\le x$, where $X$ is the random variable. 
        - Since this is a discrete distribution, $P(X\le x)=P(X < x+1)$.
        - If we wanted to calculate $P(X>x)$ then we would use `p[DIST](x,parameters,lower.tail=FALSE)`
        - If we wanted to calculate $P(X\ge x)$ then we would use `p[DIST](x-1,parameters,lower.tail=FALSE)`
    - `q[DIST](q,parameters)`
        - The value $x$ for which $P(X \le x)=q$. 
        - Since this is a continuous distribution, $q=P(X\le x)=P(X < x+1)$.
        - This function is used to calculate percentiles. 
        - If we wanted to calculate the value of $q$ for which $P(X>x)$ then we would use `q[DIST](q,parameters,lower.tail=FALSE)`
    - `r[DIST](n,parameters)`
        - Returns a list of $n$ values distributed according to `[DIST]`

Let's verify what the `d[DIST]` and `p[DIST]` functions do with the exponential and binomial distributions. I am using these distributions because we can verify most of our answers by hand. 

We'll use the following parameters

- Exponential
    - `rate=1`
- Binomial
    - `size=10`
    - `prob=0.5`

The function `d[DIST]` is the value of the probability density or probability function. For the exponential distribution we have

$$
f_{exp}(x)=\begin{cases}
\lambda e^{-\lambda x} &\quad x\ge 0\\
0 & \text{otherwise}
\end{cases}
$$

where $\lambda$ is `rate` (in our example $\lambda=1$). For the binomial we have 

$$
f_{binom}(x)=\frac{n!\pi^x(1-\pi)^{n-x}}{x!(n-x)!} \qquad \text{for }x=1,2,\ldots,n
$$

where $n$ is `size` and $\pi$ is `prob`. Let's calculate these functions when $x=2$ using the parameters we used above. 

$$
f_{exp}(2)= 1\cdot e^{-1\cdot 2}=e^{-2}\approx 0.13534
$$

and 

$$
f_{binom}(2)=\frac{10!(.5)^2(1-.5)^{10-2}}{2!(10-2)!}=\frac{10! (.5)^2(.5)^8}{2!8!}=\frac{10\cdot 9(.5)^{10}}{2}\approx 0.0439
$$

And now let's use R to calculate these numbers. In this case we will use the `d[DIST]` function. We should get the same answers.

Exponential-

In [17]:
dexp(2,1)

[1] 0.1353353

Binomial-

In [18]:
dbinom(2,10,.5)

[1] 0.04394531

The function `p[DIST]` is the proportion of values distributed less than a specific value. This is another way of saying the probability, $P(X\le x)$. For a general continuous distribution we have 

$$
P(X\le x)=\int_{-\infty}^{x}f(t)\ dt
$$

For the exponential distribution for $x>0$ we have

$$
P(X\le x)=\int_{0}^{x}\lambda e^{-\lambda t}\ dt
$$

We can solve this integral abstractly to get the Cumulative Density Function or CDF. The CDF function is defined as $\text{CDF}(x)=P(X\le x)$

$$
P(X\le x)=\int_{0}^{x}\lambda e^{-\lambda t}\ dt=-e^{-\lambda t}|_{0}^{x}=1-e^{-\lambda x}
$$

For a general discrete distribution we have 

$$
P(X\le x)=\sum_{n\le x} f(n)
$$

Solving this summation is can be quite difficult, but in some cases it is possible. 


Let's calculate the probabilities  $P(X\le 2)$ for our example functions using the parameters we used above. 

$$
P_{exp}(X\le 2)= \int_{0}^{2}\lambda e^{-\lambda t}\ dt=-e^{-\lambda t}|_{0}^{2}=1-e^{-\lambda 2}\approx 0.865
$$

and 

$$
\begin{array}{rl}
P_{binom}(X\le 2)&=\sum_{n=0}^{2}\frac{n!\pi^x(1-\pi)^{n-x}}{x!(n-x)!}\\
&=\frac{10!(.5)^0(1-.5)^{10-0}}{0!(10-0)!}+\frac{10!(.5)^1(1-.5)^{10-1}}{1!(10-1)!}+\frac{10!(.5)^2(1-.5)^{10-2}}{2!(10-2)!}\\
&=(.5)^{10}+10(.5)^1(.5)^9+ 45(.5)^{10}\\
&\approx 0.0547
\end{array}
$$

And now let's use R to calculate these probabilites. In this case we would use the `p[DIST]` function. 

Exponential -

In [19]:
pexp(2,1)

[1] 0.8646647

Binomial-

In [20]:
pbinom(2,10,.5)

[1] 0.0546875

## Applications of Probability

So we can see that R can be used to calculate probabilities and values for abstract distribution functions, but what about applying these concepts to the real world. Well we can still do this, but it will involve working a bit more with data. 

### Calculating the Chance of a Hit

One of the problems on our labs was to calculate the probability that Albert Pujols would get a hit if he was at bat 3 times. The fact that we are talking about an "on/off" type of event (in this case hit/no hit) during a limit set (in this case 3 times at bat) suggests that we should use the binomial distribution. 

In order to use the binomial distribution we have to calculate the overall probability of getting a hit. To do this we need to consider all the times Pujols got a hit when at bat.

The first thing we will do is filter the `Batting` data set to only include information about Albert Pujols. Normally we would have to look up his player id, but I know that his player id is `pujolal01`

In [28]:
pujols <- filter(Batting,playerID=="pujolal01")

Now we just divide the total of hit by the number of at bats.

In [30]:
hits <- sum(pujols$H)
at_bats<-sum(pujols$AB)
prob<-hits/at_bats
prob

[1] 0.2996164

Thus, Albert Pujols has an overall probability of 29.961% of getting a hit when at bat. Now in order to get a hit when at bat three times, he could get 1 hit, 2 hits, or 3 hits, Thus we are looking to calculate an upper tail probability. In fact we are looking for the complement of getting no hits or `1-pbinom(0,3,prob)`

In [31]:
1-pbinom(0,3,prob)

[1] 0.6564357

Thus if Albert Pujols is at bat 3 times he has a 65.64% chance of getting at least one hit.

## Contingency Tables

Another thing that we can do in R is create contingency tables. By creating contingency tables we can easily calculate conditional probabilities

### Current Population Survey
The Current Population Survey (CPS) is used to supplement census information between census years. The `CPS85` dataset consists of a random sample of persons from the 1985 Current Population Survey, with information on wages and other characteristics of the workers, including sex, number of years of education, years of work experience, occupational status, region of residence and union membership. This dataset is included in the `mosaicData` library so you will need to be sure to load this library if you want to work with this data.

Let's take a look at what is in this dataset

In [33]:
str(CPS85)

'data.frame':	534 obs. of  11 variables:
 $ wage    : num  9 5.5 3.8 10.5 15 9 9.57 15 11 5 ...
 $ educ    : int  10 12 12 12 12 16 12 14 8 12 ...
 $ race    : Factor w/ 2 levels "NW","W": 2 2 2 2 2 2 2 2 2 2 ...
 $ sex     : Factor w/ 2 levels "F","M": 2 2 1 1 2 1 1 2 2 1 ...
 $ hispanic: Factor w/ 2 levels "Hisp","NH": 2 2 2 2 2 2 2 2 2 2 ...
 $ south   : Factor w/ 2 levels "NS","S": 1 1 1 1 1 1 1 1 1 1 ...
 $ married : Factor w/ 2 levels "Married","Single": 1 1 2 1 1 1 1 2 1 1 ...
 $ exper   : int  27 20 4 29 40 27 5 22 42 14 ...
 $ union   : Factor w/ 2 levels "Not","Union": 1 1 1 1 2 1 2 1 1 1 ...
 $ age     : int  43 38 22 47 58 49 23 42 56 32 ...
 $ sector  : Factor w/ 8 levels "clerical","const",..: 2 7 7 1 2 1 8 7 4 7 ...


Let's figure out whether sex is independent of completing high school. *Please forgive the binary nature of the data. It was 1985. Although creating appropriate categories that accurately represent a population is an important factor in population demographics.* Ideally when implementing the survey there would be more categories that would accurately capture the demographics of the population. Since we are unable to change the data that was collected I will use the terms in the data, *female* and *male*, but acknowledge that the terms would probably be more accurately referred to as *assigned female at birth* and *assigned male at birth*.


### Creating a Contigency Table

First we gather numbers that would form our contingency table. In this case we want have a category for whether a person completed high school. The `educ` variable is an int which describes the highest grade completed. So if a person graduated highschool they would have `educ>=12`.

To create a table of the joint frequency of these numbers we can just use the `table` command

In [38]:
table(CPS85$sex,CPS85$educ)

   
      2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  F   0   0   0   0   2   1   9   5   5  11 110  16  23   6  34  14   9
  M   1   1   1   1   1   4   6   7  12  16 109  21  33   7  37  10  22

If we wanted the probabilities then insert this table into the `prop.table` command

In [39]:
table(CPS85$sex,CPS85$educ)%>% prop.table()

   
              2           3           4           5           6           7
  F 0.000000000 0.000000000 0.000000000 0.000000000 0.003745318 0.001872659
  M 0.001872659 0.001872659 0.001872659 0.001872659 0.001872659 0.007490637
   
              8           9          10          11          12          13
  F 0.016853933 0.009363296 0.009363296 0.020599251 0.205992509 0.029962547
  M 0.011235955 0.013108614 0.022471910 0.029962547 0.204119850 0.039325843
   
             14          15          16          17          18
  F 0.043071161 0.011235955 0.063670412 0.026217228 0.016853933
  M 0.061797753 0.013108614 0.069288390 0.018726592 0.041198502

Now we have a lot more information than we need because we have a breakdown according to highest degree awarded. What we want is whether or not the individual graduated high school. To do this we will need to `mutate` the data set to create a `highschool` column

In [49]:
CPS <- CPS85 %>%
  mutate(
    highschool = cut(educ,
      breaks = c(0, 11, 18),
      labels = c("no", "yes")
) )

In [50]:
table(CPS$sex,CPS$highschool)

   
     no yes
  F  33 212
  M  50 239

Now we have our contignecy table according to the frequencies, but let's get the probabilities. 

In [52]:
educ_cont_table <- table(CPS$sex,CPS$highschool)%>% prop.table()
educ_cont_table

   
            no        yes
  F 0.06179775 0.39700375
  M 0.09363296 0.44756554

Now we can calulate the conditional probabilities 

$$
P(\text{ High School Graduate } | \text{ Female })=\frac{P(\text{ High School Graduate and Female })}{P(\text{ Female })}
$$

In [85]:
grad_given_F <- educ_cont_table[1,2]/(educ_cont_table[1,1]+educ_cont_table[1,2])
grad_given_F

[1] 0.8653061

Thus there is an 86.53% chance that a person would graduate high school in 1985 given that they were female. 

$$
P(\text{ Female } | \text{ High School Graduate })=\frac{P(\text{ High School Graduate and Female })}{P(\text{ High School Graduate })}
$$

In [86]:
F_given_grad <- educ_cont_table[1,2]/(educ_cont_table[1,2]+educ_cont_table[2,2])
F_given_grad

[1] 0.4700665

Thus there is an 47.01% chance that a person is female given that they were a high school graduate in 1985. 

If female and graduating high school are independent then 

$$
P(\text{ High School Graduate and Female })=P(\text{ High School Graduate} )P(\text{ Female })
$$


In [84]:
educ_cont_table[1,2]

[1] 0.3970037

In [113]:
female<-educ_cont_table[1,1]+educ_cont_table[1,2]
highschool<-educ_cont_table[1,2]+educ_cont_table[2,2]
female*highschool

[1] 0.3874897

These numbers appear to be close but not equal. If we were working with the population then we could conclude that graduating high school and being female isn't indepdent within the population, but the Current Population survey is only a sample. This is where we would need to formulate a test for independence.


### Chi-Square Test For Independence

One such test for independence between variables is called the chi-square test. The idea of this test is to first assume that the two variables are independent. Then we calculate the probability that we would see the specific data the we have given that the two variables are independent. This probability is called the *p-value* for the test. If this probability is low (normally less than 5%), then it is unlikely that we would see the data if the variables were independent. Thus we could say with a certain confidence that the variables are **not** independent. This is one version of a hypothesis test, which we will be learning more about later on.

Now how do we calculate this p-value. For each type of hypothesis test there is a different way to calculate the p-value. Here we will go over how to do it in the case of the chi-square test.

#### Calculating the p-value

The Chi-square test of independence works by comparing the observed counts to the expected counts. Therefore, our task is to derive the expected table containing the expected counts from the observed table. The expected table is what we expect the contignecy table to look like if the two categorical variables are independent. From probability theory, we know that two events are said to be independent if their joint probability is equal to the product of their conditional probabilities. We use this fact to construct a new contingency table with our expected counts. 

Once we have this new table we calculate

$$
\chi^2=\sum\frac{(\text{observed count}-\text{expected count})^2}{\text{expected count}}
$$

We would then use a chi-square distribution table to find the p-value from this number. Luckily, R will do this whole process for us relatively easily.

In [100]:
chisq.test(CPS$sex,CPS$highschool)


	Pearson's Chi-squared test with Yates' continuity correction

data:  CPS$sex and CPS$highschool
X-squared = 1.2054, df = 1, p-value = 0.2722


Here we have that $\chi^2=1.2054$ and our p-value is $0.2722$. Since our p-value is not small (again, less than 5%) we can't say for certain whether these two variables are independent or not. A p-value higher than 5% doesn't support our assumption that the variables are independent it only prevents us from negating this assumption. We still can't say for certain.